In [41]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import datetime as dt
from random import seed
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import backend as K
tf.compat.v1.experimental.output_all_intermediates(True)

## 1. Using only rainfall and water level inputs

In [42]:
seed(36)

In [43]:
df = pd.read_csv("compiled_data_2016_2017.csv")
df.head(10)

,Unnamed: 0,index,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Waterlevel_Montalban,Discharge_Sto_Nino,Discharge_San_Jose,Cross_Section_Sto_Nino,Cross_Section_Montalban,Velocity_Sto_Nino,Velocity_Montalban,datetime,t,x
0,0,0,0,1,2,0,0,12.18,21.03,21.033407,14.842428,803.88,630.9,0.026165,0.023526,2016-01-01 00:00:00,0.0,14420
1,1,1,0,1,1,1,0,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 01:00:00,3600.0,14420
2,2,2,1,1,1,0,1,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 02:00:00,7200.0,14420
3,3,3,0,0,0,1,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 03:00:00,10800.0,14420
4,4,4,1,1,1,0,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 04:00:00,14400.0,14420
5,5,5,0,0,1,0,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 05:00:00,18000.0,14420
6,6,6,0,1,1,0,0,12.21,21.03,21.780375,14.842428,805.86,630.9,0.027027,0.023526,2016-01-01 06:00:00,21600.0,14420
7,7,7,0,0,0,0,0,12.21,21.03,21.780375,14.842428,805.86,630.9,0.027027,0.023526,2016-01-01 07:00:00,25200.0,14420
8,8,8,1,0,1,0,0,12.21,21.03,21.780375,14.842428,805.86,630.9,0.027027,0.023526,2016-01-01 08:00:00,28800.0,14420
9,9,9,0,0,0,0,0,12.21,21.03,21.780375,14.842428,805.86,630.9,0.027027,0.023526,2016-01-01 09:00:00,32400.0,14420


In [ ]:
df = df[['Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'Waterlevel_Sto_Nino']]
df.head(10)

In [ ]:
# Splitting for time series: split into 50-25-25
n = len(df)
train_df = df[0:int(n*0.5)]
val_df = df[int(n*0.5):int(n*0.75)]
test_df = df[int(n*0.75):]

In [ ]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns], axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels
    
    # Creating tf datasets for more convenient use and integration into model in the future
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)

        ds = ds.map(self.split_window)

        return ds
    
    # properties to access them as tf datasets
    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result

In [ ]:
# The wide window uses independent hours of data as input to predict the water level of the next hour
# Here, the prediction is done on 6 hours
# This is used for Dense and Recurrent Neural Networks
wide_window = WindowGenerator(
        input_width=6, label_width=6, shift=1,
        label_columns=['Waterlevel_Sto_Nino']
    )

wide_window

In [ ]:
# The conv window is used for the Convolutional Neural Netwrok
# 6 consecutive hours of data are used together to make predictions one hour into the future
CONV_WIDTH = 6
conv_window = WindowGenerator(
        input_width=CONV_WIDTH,
        label_width=1,
        shift=1,
        label_columns=['Waterlevel_Sto_Nino']
    )

conv_window

In [44]:
def r_square(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.square(K.sum(xm * ym))
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = (x_square_sum * y_square_sum) + K.epsilon()
    
    r = r_num / r_den
    return r

In [45]:
def NSE(y_true, y_pred):
    '''
    This is the Nash-Sutcliffe Efficiency Coefficient
    '''
    y_pred = K.flatten(y_pred)
    y_true = K.flatten(y_true)

    
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
# For easy compiling and fitting of different models
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(), 
        optimizer='adam', 
        metrics=[tf.keras.metrics.MeanSquaredError(), NSE, r_square]
    )

    history = model.fit(
        window.train, 
        epochs=MAX_EPOCHS,
        validation_data=window.val,
        callbacks=[early_stopping]
    )

    return history

In [ ]:
# Dense Neural Network
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

# Convolution Neural Network
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=(CONV_WIDTH,), activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1),
])

# LSTM
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

In [ ]:
dense_history = compile_and_fit(dense, wide_window)

In [ ]:
dense_history.history

In [ ]:
conv_history = compile_and_fit(conv_model, conv_window)

In [ ]:
lstm_history = compile_and_fit(lstm_model, wide_window)

In [ ]:
val_performance = {}
performance = {}

In [ ]:
val_performance['Dense'] = dense.evaluate(wide_window.val)

In [ ]:
performance['Dense'] = dense.evaluate(wide_window.test, verbose=0)

In [ ]:
val_performance['Conv'] = conv_model.evaluate(conv_window.val)

In [ ]:
performance['Conv'] = conv_model.evaluate(conv_window.test, verbose=0)

In [ ]:
val_performance['LSTM'] = lstm_model.evaluate(wide_window.val)

In [ ]:
performance['LSTM'] = lstm_model.evaluate(wide_window.test, verbose=0)

In [ ]:
val_performance

In [ ]:
performance

In [ ]:
import pandas as pd



df_val = pd.DataFrame.from_dict(performance, orient='index', columns=['Loss', 'MSE', 'NSE', 'R^2'])

In [ ]:
df_val

## 2. Univariate Time series ANN - Using only water level 


In [ ]:
import tensorflow as tf
import numpy as np

class WindowGenerator():
    def __init__(self, input_width, label_width, shift, train_df=train_df, val_df=val_df, test_df=test_df):
        # Store the raw data - select only the Sto Nino water level column
        self.train_df = train_df[['Waterlevel_Sto_Nino']].values
        self.val_df = val_df[['Waterlevel_Sto_Nino']].values
        self.test_df = test_df[['Waterlevel_Sto_Nino']].values

        # Work out the window parameters
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift
        self.total_window_size = input_width + shift

        # Input and label slices
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]
        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}'])

    def split_window(self, features):
        # Features shape is (batch, time_steps, 1)
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        
        # Set shapes explicitly
        inputs.set_shape([None, self.input_width, 1])
        labels.set_shape([None, self.label_width, 1])
        
        return inputs, labels
    
    def make_dataset(self, data):
        # Ensure data is float32 and has correct shape (samples, 1)
        data = np.array(data, dtype=np.float32)
        if len(data.shape) == 1:
            data = data.reshape(-1, 1)
            
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)
        
        ds = ds.map(self.split_window)
        return ds
    
    @property
    def train(self):
        return self.make_dataset(self.train_df)
    
    @property
    def val(self):
        return self.make_dataset(self.val_df)
    
    @property
    def test(self):
        return self.make_dataset(self.test_df)
    
    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            result = next(iter(self.train))
            self._example = result
        return result

In [ ]:
# The wide window uses independent hours of data as input to predict the water level of the next hour
# Here, the prediction is done on 6 hours
# This is used for Dense and Recurrent Neural Networks
wide_window = WindowGenerator(
        input_width=6, label_width=6, shift=1,
        train_df=train_df,
        val_df=val_df,
        test_df=test_df
    )

wide_window

In [ ]:
# The conv window is used for the Convolutional Neural Netwrok
# 6 consecutive hours of data are used together to make predictions one hour into the future
CONV_WIDTH = 6
conv_window = WindowGenerator(
        input_width=CONV_WIDTH,
        label_width=1,
        shift=1,
        train_df=train_df,
        val_df=val_df,
        test_df=test_df
    )

conv_window

In [ ]:
# Dense Neural Network
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', input_shape=[wide_window.input_width, 1]),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

# Convolution Neural Network
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=(CONV_WIDTH,), activation='relu', input_shape=[conv_window.input_width, 1]),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1),
])

# LSTM
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True, input_shape=[wide_window.input_width, 1]),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

In [ ]:
def r_square(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    r_num = K.square(K.sum(xm * ym))
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = (x_square_sum * y_square_sum) + K.epsilon()
    
    r = r_num / r_den
    return r

In [ ]:
def NSE(y_true, y_pred):
    '''
    This is the Nash-Sutcliffe Efficiency Coefficient
    '''
    y_pred = K.flatten(y_pred)
    y_true = K.flatten(y_true)

    
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
# For easy compiling and fitting of different models
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(), 
        optimizer='adam', 
        metrics=[tf.keras.metrics.MeanSquaredError(), NSE, r_square]
    )

    history = model.fit(
        window.train, 
        epochs=MAX_EPOCHS,
        validation_data=window.val,
        callbacks=[early_stopping]
    )

    return history

In [ ]:
dense_history = compile_and_fit(dense, wide_window)

In [ ]:
conv_history = compile_and_fit(conv_model, conv_window)

In [ ]:
lstm_history = compile_and_fit(lstm_model, wide_window)

In [ ]:
val_performance = {}
performance = {}

In [ ]:
val_performance['Dense'] = dense.evaluate(wide_window.val)

In [ ]:
performance['Dense'] = dense.evaluate(wide_window.test, verbose=0)

In [ ]:
val_performance['Conv'] = conv_model.evaluate(conv_window.val)

In [ ]:
performance['Conv'] = conv_model.evaluate(conv_window.test, verbose=0)

In [ ]:
val_performance['LSTM'] = lstm_model.evaluate(wide_window.val)

In [ ]:
performance['LSTM'] = lstm_model.evaluate(wide_window.test, verbose=0)

In [ ]:
performance

In [ ]:
pd.DataFrame.from_dict(performance, orient='index', columns=['Loss', 'MSE', 'NSE', 'R^2'])

In [ ]:
# Predictions

# Get the last input sequence from the test dataset
def get_last_input_sequence(test_data, input_width):
    """
    Extracts the last input sequence for prediction
    
    Args:
    - test_data: Test dataset array
    - input_width: Number of time steps for input
    
    Returns:
    - Last input sequence reshaped for model prediction
    """
    last_sequence = test_data[-input_width:]
    return last_sequence.reshape(1, input_width, 1)

# Predict future steps
def predict_future(model, initial_input, num_steps):
    """
    Predict future time steps recursively
    
    Args:
    - model: Trained Keras model
    - initial_input: Initial input sequence (shape: [1, input_width, 1])
    - num_steps: Number of future time steps to predict
    
    Returns:
    - Predicted future values
    """
    current_input = initial_input
    predictions = []
    
    for _ in range(num_steps):
        # Predict next time step
        prediction = model.predict(current_input)
        print(prediction[0,-1,0])
        
        # Append prediction
        predictions.append(prediction[0, -1, 0])

        # Slide window: remove oldest input, append new prediction
        current_input = np.roll(current_input, -1, axis=1)

        current_input[0, -1, 0] = prediction[0, -1, 0]

        
    
    return np.array(predictions)

# Assuming test_df is your test dataframe and wide_window is your WindowGenerator
last_input = get_last_input_sequence(wide_window.test_df, input_width=6)
future_predictions = predict_future(lstm_model, last_input, num_steps=24)

In [ ]:
future_predictions

In [ ]:
test_df['Waterlevel_Sto_Nino']

## 3. ANN with inputs: rainfall, manning's coefficient, bed slope, time, discharge 

outputs: water level in sto nino, water velocity in sto nino 

Note: window generator was not used

results 4.4.1

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

In [47]:
df_2016_2017 = pd.read_csv("compiled_data_2016_2017.csv")

df_2016_2017['friction_coeff'] = [0.033 for i in range(len(df_2016_2017))]
df_2016_2017['slope'] = [1/1500 for i in range(len(df_2016_2017))]

In [48]:
df_2016_2017

,Unnamed: 0,index,Rainfall_Aries,Rainfall_Boso,Rainfall_Campana,Rainfall_Nangka,Rainfall_Oro,Waterlevel_Sto_Nino,Waterlevel_Montalban,Discharge_Sto_Nino,Discharge_San_Jose,Cross_Section_Sto_Nino,Cross_Section_Montalban,Velocity_Sto_Nino,Velocity_Montalban,datetime,t,x,friction_coeff,slope
0,0,0,0,1,2,0,0,12.18,21.03,21.033407,14.842428,803.88,630.9,0.026165,0.023526,2016-01-01 00:00:00,0.0,14420,0.033,0.000667
1,1,1,0,1,1,1,0,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 01:00:00,3600.0,14420,0.033,0.000667
2,2,2,1,1,1,0,1,12.19,21.03,21.280072,14.842428,804.54,630.9,0.026450,0.023526,2016-01-01 02:00:00,7200.0,14420,0.033,0.000667
3,3,3,0,0,0,1,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 03:00:00,10800.0,14420,0.033,0.000667
4,4,4,1,1,1,0,0,12.20,21.03,21.529056,14.842428,805.20,630.9,0.026738,0.023526,2016-01-01 04:00:00,14400.0,14420,0.033,0.000667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,17515,16059,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 19:00:00,63140400.0,14420,0.033,0.000667
17516,17516,16424,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 20:00:00,63144000.0,14420,0.033,0.000667
17517,17517,16789,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 21:00:00,63147600.0,14420,0.033,0.000667
17518,17518,17154,0,0,0,0,0,12.44,21.18,28.244204,17.224575,821.04,635.4,0.034401,0.027108,2017-12-31 22:00:00,63151200.0,14420,0.033,0.000667


In [49]:
train_2016_2017 = df_2016_2017[:int(0.50*len(df_2016_2017))]
val_2016_2017 = df_2016_2017[int(0.50*len(df_2016_2017)):int(0.75*len(df_2016_2017))]
test_2016_2017 = df_2016_2017[int(0.75*len(df_2016_2017)):]

In [99]:
# all inputs
X_train_2016_2017 = np.array(train_2016_2017[['Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'x' ,'t', 'Discharge_Sto_Nino', 'friction_coeff', 'slope']].values.tolist())
X_val_2016_2017 = np.array(val_2016_2017[['Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'x','t', 'Discharge_Sto_Nino', 'friction_coeff', 'slope']].values.tolist())
X_test_2016_2017 = np.array(test_2016_2017[['Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'x','t', 'Discharge_Sto_Nino', 'friction_coeff', 'slope']].values.tolist())
Y_train_2016_2017 = np.array(train_2016_2017[['Velocity_Sto_Nino','Waterlevel_Sto_Nino']].values.tolist())
Y_val_2016_2017 = np.array(val_2016_2017[['Velocity_Sto_Nino','Waterlevel_Sto_Nino']].values.tolist())
Y_test_2016_2017 = np.array(test_2016_2017[['Velocity_Sto_Nino','Waterlevel_Sto_Nino']].values.tolist())

In [112]:
Y_train_2016_2017.shape

(8760, 2)

In [59]:
CONV_WIDTH = 6
# Dense Neural Network
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

# Convolution Neural Network
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=(CONV_WIDTH,), activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1),
])

# LSTM
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=1)
])

In [60]:
def custom_loss(y_true, y_pred):
    velocity_loss = tf.keras.losses.mean_squared_error(y_true[:, 0], y_pred[:, 0])
    waterlevel_loss = tf.keras.losses.mean_squared_error(y_true[:, 1], y_pred[:, 1])
    return velocity_loss + waterlevel_loss  # or any other combination

In [73]:
def create_train_dnn_model(X_train, y_train, X_val, y_val, max_epochs=20, patience=2):
    """
    Creates, compiles, and trains a Deep Neural Network model.
    
    Parameters:
    X_train (array-like): Training input data
    y_train (array-like): Training target data
    X_val (array-like): Validation input data
    y_val (array-like): Validation target data
    max_epochs (int): Maximum number of training epochs
    patience (int): Number of epochs with no improvement after which training will be stopped
    
    Returns:
    tuple: (model, history)
    """
    # Create the model
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dense(2, activation='linear')  # 2 outputs: velocity and water level
    ])
    
    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        mode='min'
    )
    
    # Compile the model
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError(), NSE, r_square]
    )
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=max_epochs,
        callbacks=[early_stopping]
    )
    
    return model, history



In [117]:
def create_train_cnn_model(X_train, y_train, X_val, y_val, max_epochs=20, patience=2):
    # Reshape input for 1D CNN
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
    
    conv_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(10, 1)),
        tf.keras.layers.Conv1D(filters=64, kernel_size=6, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(units=2)
    ])
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        mode='min'
    )
    
    conv_model.compile(
        optimizer='adam',
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError(), NSE, r_square]
    )
    
    conv_history = conv_model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=max_epochs,
        callbacks=[early_stopping]
    )
    
    return conv_model, conv_history

In [135]:
def create_train_lstm_model(X_train, y_train, X_val, y_val, max_epochs=20, patience=2):
    """
    Creates, compiles, and trains a Deep Neural Network model.
    
    Parameters:
    X_train (array-like): Training input data
    y_train (array-like): Training target data
    X_val (array-like): Validation input data
    y_val (array-like): Validation target data
    max_epochs (int): Maximum number of training epochs
    patience (int): Number of epochs with no improvement after which training will be stopped
    
    Returns:
    tuple: (model, history)
    """
    
    if len(X_train.shape) == 2:
        X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    if len(X_val.shape) == 2:
        X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
        
    
        
    lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    tf.keras.layers.Flatten(),  # Add Flatten layer to handle dimension mismatch
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2)
    ])

    
    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        mode='min'
    )
    
    # Compile the model
    lstm_model.compile(
        optimizer='adam',
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError(), NSE, r_square]
    )
    
    # Train the model
    lstm_history = lstm_model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=max_epochs,
        callbacks=[early_stopping]
    )
    
    return lstm_model, lstm_history

In [105]:

# Train the model
dnn_model, dnn_history = create_train_dnn_model(
    X_train=X_train_2016_2017,
    y_train=Y_train_2016_2017,
    X_val=X_val_2016_2017,
    y_val=Y_val_2016_2017,
    max_epochs=20,
    patience=2
)






Epoch 1/20
274/274 [==============================] - 2s 2ms/step - loss: 97851162624.0000 - mean_squared_error: 97851162624.0000 - NSE: -2536450560.0000 - r_square: 0.1624 - val_loss: 4152266.2500 - val_mean_squared_error: 4152266.2500 - val_NSE: -117938.8594 - val_r_square: 0.0148
Epoch 2/20
274/274 [==============================] - 0s 2ms/step - loss: 1485402.2500 - mean_squared_error: 1485402.2500 - NSE: -38811.5781 - r_square: 0.0411 - val_loss: 5970993.0000 - val_mean_squared_error: 5970993.0000 - val_NSE: -169467.7812 - val_r_square: 0.0132
Epoch 3/20
274/274 [==============================] - 0s 2ms/step - loss: 1458505.8750 - mean_squared_error: 1458505.8750 - NSE: -38101.3984 - r_square: 0.0412 - val_loss: 3443729.5000 - val_mean_squared_error: 3443729.5000 - val_NSE: -97841.9297 - val_r_square: 0.0168
Epoch 4/20
274/274 [==============================] - 0s 2ms/step - loss: 1458348.7500 - mean_squared_error: 1458348.7500 - NSE: -38091.1914 - r_square: 0.0430 - val_loss: 231

In [120]:
# Train the model
cnn_model, cnn_history = create_train_cnn_model(
    X_train=X_train_2016_2017,
    y_train=Y_train_2016_2017,
    X_val=X_val_2016_2017,
    y_val=Y_val_2016_2017,
    max_epochs=20,
    patience=2 )

Epoch 1/20
274/274 [==============================] - 2s 3ms/step - loss: 2700339200.0000 - mean_squared_error: 2700339200.0000 - NSE: -70116400.0000 - r_square: 0.2346 - val_loss: 73398.1641 - val_mean_squared_error: 73398.1641 - val_NSE: -2070.6479 - val_r_square: 0.0060
Epoch 2/20
274/274 [==============================] - 1s 2ms/step - loss: 401242.9062 - mean_squared_error: 401242.9062 - NSE: -10380.2891 - r_square: 0.2970 - val_loss: 9855.9053 - val_mean_squared_error: 9855.9053 - val_NSE: -278.9723 - val_r_square: 0.0454
Epoch 3/20
274/274 [==============================] - 1s 2ms/step - loss: 7824.5405 - mean_squared_error: 7824.5405 - NSE: -203.1162 - r_square: 0.3363 - val_loss: 11610.0244 - val_mean_squared_error: 11610.0244 - val_NSE: -328.4637 - val_r_square: 0.0469
Epoch 4/20
274/274 [==============================] - 1s 2ms/step - loss: 3925.5723 - mean_squared_error: 3925.5723 - NSE: -101.3924 - r_square: 0.3419 - val_loss: 1557.0587 - val_mean_squared_error: 1557.0587 

In [116]:
print("X_train shape:", X_train_2016_2017.shape)
print("Y_train shape:", Y_train_2016_2017.shape)

X_train shape: (8760, 10)
Y_train shape: (8760, 2)


In [136]:
# Train the model
lstm_model, lstm_history = create_train_lstm_model(
    X_train=X_train_2016_2017,
    y_train=Y_train_2016_2017,
    X_val=X_val_2016_2017,
    y_val=Y_val_2016_2017,
    max_epochs=20,
    patience=2
)

Epoch 1/20
274/274 [==============================] - 3s 5ms/step - loss: 8.5357 - mean_squared_error: 8.5357 - NSE: 0.7777 - r_square: 0.0022 - val_loss: 0.1313 - val_mean_squared_error: 0.1313 - val_NSE: 0.9962 - val_r_square: 1.0573e-12
Epoch 2/20
274/274 [==============================] - 1s 3ms/step - loss: 0.2351 - mean_squared_error: 0.2351 - NSE: 0.9939 - r_square: 0.0038 - val_loss: 0.0893 - val_mean_squared_error: 0.0893 - val_NSE: 0.9975 - val_r_square: 4.8293e-07
Epoch 3/20
274/274 [==============================] - 1s 3ms/step - loss: 0.2327 - mean_squared_error: 0.2327 - NSE: 0.9940 - r_square: 2.3889e-04 - val_loss: 0.0715 - val_mean_squared_error: 0.0715 - val_NSE: 0.9980 - val_r_square: 2.6867e-12
Epoch 4/20
274/274 [==============================] - 1s 3ms/step - loss: 0.2303 - mean_squared_error: 0.2303 - NSE: 0.9940 - r_square: 2.4332e-04 - val_loss: 0.1788 - val_mean_squared_error: 0.1788 - val_NSE: 0.9949 - val_r_square: 1.2194e-07
Epoch 5/20
274/274 [============

In [121]:
dnn_model.evaluate(X_test_2016_2017, Y_test_2016_2017)

137/137 [==============================] - 0s 1ms/step - loss: 22348358.0000 - mean_squared_error: 22348358.0000 - NSE: -564409.0000 - r_square: 0.0569


[22348358.0, 22348358.0, -564409.0, 0.05692477896809578]

In [123]:
X_test_2016_2017 = X_test_2016_2017.reshape(X_test_2016_2017.shape[0], X_test_2016_2017.shape[1], 1)

cnn_model.evaluate(X_test_2016_2017, Y_test_2016_2017)

137/137 [==============================] - 0s 1ms/step - loss: 46835832.0000 - mean_squared_error: 46835832.0000 - NSE: -1183620.5000 - r_square: 0.0553


[46835832.0, 46835832.0, -1183620.5, 0.055303093045949936]

In [139]:
X_test_2016_2017 = np.array(test_2016_2017[['Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'x','t', 'Discharge_Sto_Nino', 'friction_coeff', 'slope']].values.tolist())
if len(X_test_2016_2017.shape) == 2:
    X_test_2016_2017 = X_test_2016_2017.reshape((X_test_2016_2017.shape[0], 1, X_test_2016_2017.shape[1]))
lstm_model.evaluate(X_test_2016_2017, Y_test_2016_2017)

137/137 [==============================] - 0s 1ms/step - loss: 0.2298 - mean_squared_error: 0.2298 - NSE: 0.9949 - r_square: 3.7171e-13


[0.22978727519512177,
 0.22978727519512177,
 0.9948520064353943,
 3.7171481170883425e-13]

### Without "t"

In [164]:
X_train_2016_2017 = np.array(train_2016_2017[['Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'x' , 'Discharge_Sto_Nino', 'friction_coeff', 'slope']].values.tolist())
X_val_2016_2017 = np.array(val_2016_2017[['Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'x', 'Discharge_Sto_Nino', 'friction_coeff', 'slope']].values.tolist())
X_test_2016_2017 = np.array(test_2016_2017[['Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'x', 'Discharge_Sto_Nino', 'friction_coeff', 'slope']].values.tolist())
Y_train_2016_2017 = np.array(train_2016_2017[['Velocity_Sto_Nino','Waterlevel_Sto_Nino']].values.tolist())
Y_val_2016_2017 = np.array(val_2016_2017[['Velocity_Sto_Nino','Waterlevel_Sto_Nino']].values.tolist())
Y_test_2016_2017 = np.array(test_2016_2017[['Velocity_Sto_Nino','Waterlevel_Sto_Nino']].values.tolist())

In [165]:
def create_train_dnn_model(X_train, y_train, X_val, y_val, max_epochs=20, patience=2):
    """
    Creates, compiles, and trains a Deep Neural Network model.
    
    Parameters:
    X_train (array-like): Training input data
    y_train (array-like): Training target data
    X_val (array-like): Validation input data
    y_val (array-like): Validation target data
    max_epochs (int): Maximum number of training epochs
    patience (int): Number of epochs with no improvement after which training will be stopped
    
    Returns:
    tuple: (model, history)
    """
    # Create the model
    model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dense(2, activation='linear')  # 2 outputs: velocity and water level
    ])
    
    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        mode='min'
    )
    
    # Compile the model
    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError(), NSE, r_square]
    )
    
    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=max_epochs,
        callbacks=[early_stopping]
    )
    
    return model, history



In [166]:
def create_train_cnn_model(X_train, y_train, X_val, y_val, max_epochs=20, patience=2):
    # Reshape input for 1D CNN
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
    
    conv_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(9, 1)),
        tf.keras.layers.Conv1D(filters=64, kernel_size=6, activation='relu'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(units=2)
    ])
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        mode='min'
    )
    
    conv_model.compile(
        optimizer='adam',
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError(), NSE, r_square]
    )
    
    conv_history = conv_model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=max_epochs,
        callbacks=[early_stopping]
    )
    
    return conv_model, conv_history

In [167]:
def create_train_lstm_model(X_train, y_train, X_val, y_val, max_epochs=20, patience=2):
    """
    Creates, compiles, and trains a Deep Neural Network model.
    
    Parameters:
    X_train (array-like): Training input data
    y_train (array-like): Training target data
    X_val (array-like): Validation input data
    y_val (array-like): Validation target data
    max_epochs (int): Maximum number of training epochs
    patience (int): Number of epochs with no improvement after which training will be stopped
    
    Returns:
    tuple: (model, history)
    """
    
    if len(X_train.shape) == 2:
        X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    if len(X_val.shape) == 2:
        X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
        
    
        
    lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    tf.keras.layers.Flatten(),  # Add Flatten layer to handle dimension mismatch
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=2)
    ])

    
    # Define early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=patience,
        mode='min'
    )
    
    # Compile the model
    lstm_model.compile(
        optimizer='adam',
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError(), NSE, r_square]
    )
    
    # Train the model
    lstm_history = lstm_model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=max_epochs,
        callbacks=[early_stopping]
    )
    
    return lstm_model, lstm_history

In [168]:

# Train the model
dnn_model, dnn_history = create_train_dnn_model(
    X_train=X_train_2016_2017,
    y_train=Y_train_2016_2017,
    X_val=X_val_2016_2017,
    y_val=Y_val_2016_2017,
    max_epochs=20,
    patience=2
)






Epoch 1/20
274/274 [==============================] - 1s 2ms/step - loss: 1914.3878 - mean_squared_error: 1914.3878 - NSE: -49.1645 - r_square: 0.1261 - val_loss: 6.5735 - val_mean_squared_error: 6.5735 - val_NSE: 0.8130 - val_r_square: 0.0175
Epoch 2/20
274/274 [==============================] - 0s 2ms/step - loss: 13.1151 - mean_squared_error: 13.1151 - NSE: 0.6663 - r_square: 0.0750 - val_loss: 0.2962 - val_mean_squared_error: 0.2962 - val_NSE: 0.9916 - val_r_square: 0.0244
Epoch 3/20
274/274 [==============================] - 0s 2ms/step - loss: 3.4577 - mean_squared_error: 3.4577 - NSE: 0.9113 - r_square: 0.1385 - val_loss: 0.1324 - val_mean_squared_error: 0.1324 - val_NSE: 0.9963 - val_r_square: 0.0224
Epoch 4/20
274/274 [==============================] - 0s 2ms/step - loss: 0.9923 - mean_squared_error: 0.9923 - NSE: 0.9745 - r_square: 0.2439 - val_loss: 0.6002 - val_mean_squared_error: 0.6002 - val_NSE: 0.9829 - val_r_square: 0.0399
Epoch 5/20
274/274 [==========================

In [169]:
# Train the model
cnn_model, cnn_history = create_train_cnn_model(
    X_train=X_train_2016_2017,
    y_train=Y_train_2016_2017,
    X_val=X_val_2016_2017,
    y_val=Y_val_2016_2017,
    max_epochs=20,
    patience=2 )

Epoch 1/20
274/274 [==============================] - 1s 3ms/step - loss: 15734.3877 - mean_squared_error: 15734.3877 - NSE: -408.9473 - r_square: 0.6374 - val_loss: 0.1656 - val_mean_squared_error: 0.1656 - val_NSE: 0.9954 - val_r_square: 0.1010
Epoch 2/20
274/274 [==============================] - 1s 2ms/step - loss: 1.1951 - mean_squared_error: 1.1951 - NSE: 0.9697 - r_square: 0.5820 - val_loss: 0.2835 - val_mean_squared_error: 0.2835 - val_NSE: 0.9922 - val_r_square: 0.0863
Epoch 3/20
274/274 [==============================] - 1s 2ms/step - loss: 0.3316 - mean_squared_error: 0.3316 - NSE: 0.9915 - r_square: 0.6645 - val_loss: 0.0616 - val_mean_squared_error: 0.0616 - val_NSE: 0.9983 - val_r_square: 0.0784
Epoch 4/20
274/274 [==============================] - 0s 2ms/step - loss: 0.2429 - mean_squared_error: 0.2429 - NSE: 0.9937 - r_square: 0.7174 - val_loss: 0.0513 - val_mean_squared_error: 0.0513 - val_NSE: 0.9986 - val_r_square: 0.0806
Epoch 5/20
274/274 [=========================

In [170]:
print("X_train shape:", X_train_2016_2017.shape)
print("Y_train shape:", Y_train_2016_2017.shape)

X_train shape: (8760, 9)
Y_train shape: (8760, 2)


In [171]:
# Train the model
lstm_model, lstm_history = create_train_lstm_model(
    X_train=X_train_2016_2017,
    y_train=Y_train_2016_2017,
    X_val=X_val_2016_2017,
    y_val=Y_val_2016_2017,
    max_epochs=20,
    patience=2
)

Epoch 1/20
274/274 [==============================] - 3s 5ms/step - loss: 7.7471 - mean_squared_error: 7.7471 - NSE: 0.7984 - r_square: 0.0048 - val_loss: 0.1288 - val_mean_squared_error: 0.1288 - val_NSE: 0.9963 - val_r_square: 8.7892e-12
Epoch 2/20
274/274 [==============================] - 1s 3ms/step - loss: 0.2274 - mean_squared_error: 0.2274 - NSE: 0.9941 - r_square: 6.9606e-06 - val_loss: 0.1267 - val_mean_squared_error: 0.1267 - val_NSE: 0.9964 - val_r_square: 2.8278e-12
Epoch 3/20
274/274 [==============================] - 1s 3ms/step - loss: 0.2282 - mean_squared_error: 0.2282 - NSE: 0.9941 - r_square: 4.4230e-06 - val_loss: 0.1387 - val_mean_squared_error: 0.1387 - val_NSE: 0.9960 - val_r_square: 1.1879e-07
Epoch 4/20
274/274 [==============================] - 1s 3ms/step - loss: 0.2276 - mean_squared_error: 0.2276 - NSE: 0.9941 - r_square: 4.5478e-06 - val_loss: 0.1375 - val_mean_squared_error: 0.1375 - val_NSE: 0.9961 - val_r_square: 1.1839e-07


In [175]:
X_test_2016_2017 = np.array(test_2016_2017[['Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'x', 'Discharge_Sto_Nino', 'friction_coeff', 'slope']].values.tolist())

dnn_model.evaluate(X_test_2016_2017, Y_test_2016_2017)

137/137 [==============================] - 0s 1ms/step - loss: 1.6420 - mean_squared_error: 1.6420 - NSE: 0.9594 - r_square: 0.1800


[1.6420223712921143,
 1.6420223712921143,
 0.9594290256500244,
 0.18002846837043762]

In [176]:
X_test_2016_2017 = X_test_2016_2017.reshape(X_test_2016_2017.shape[0], X_test_2016_2017.shape[1], 1)

cnn_model.evaluate(X_test_2016_2017, Y_test_2016_2017)

137/137 [==============================] - 0s 1ms/step - loss: 0.1322 - mean_squared_error: 0.1322 - NSE: 0.9967 - r_square: 0.3573


[0.13223299384117126,
 0.13223299384117126,
 0.9967476725578308,
 0.3573175370693207]

In [177]:
X_test_2016_2017 = np.array(test_2016_2017[['Rainfall_Aries', 'Rainfall_Boso', 'Rainfall_Campana', 'Rainfall_Nangka', 'Rainfall_Oro', 'x', 'Discharge_Sto_Nino', 'friction_coeff', 'slope']].values.tolist())
if len(X_test_2016_2017.shape) == 2:
    X_test_2016_2017 = X_test_2016_2017.reshape((X_test_2016_2017.shape[0], 1, X_test_2016_2017.shape[1]))
lstm_model.evaluate(X_test_2016_2017, Y_test_2016_2017)

137/137 [==============================] - 0s 1ms/step - loss: 0.1832 - mean_squared_error: 0.1832 - NSE: 0.9959 - r_square: 4.6070e-06


[0.18316765129566193,
 0.18316765129566193,
 0.995904266834259,
 4.607012670021504e-06]

## PINNs with window

In [ ]:
X_train_2016_2017_pinn = train_2016_2017[['x','t', 'Discharge_Sto_Nino', 'friction_coeff', 'slope']]
X_val_2016_2017_pinn = val_2016_2017[['x','t', 'Discharge_Sto_Nino', 'friction_coeff', 'slope']]
X_test_2016_2017_pinn = test_2016_2017[['x','t', 'Discharge_Sto_Nino', 'friction_coeff', 'slope']]
Y_train_2016_2017_pinn = train_2016_2017[['Velocity_Sto_Nino','Waterlevel_Sto_Nino']]
Y_val_2016_2017_pinn = val_2016_2017[['Velocity_Sto_Nino','Waterlevel_Sto_Nino']]
Y_test_2016_2017_pinn = test_2016_2017[['Velocity_Sto_Nino','Waterlevel_Sto_Nino']]

In [ ]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift, 
                 train_data, val_data, test_data):
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift
        self.total_window_size = input_width + shift
    
    def make_dataset(self, data):
        X, Y = [], []
        for i in range(len(data) - self.total_window_size + 1):
            inputs = data[i:i+self.input_width]
            labels = data[i+self.input_width:i+self.total_window_size]
            X.append(inputs)
            Y.append(labels)
        
        return np.array(X), np.array(Y)
    
    @property
    def train(self):
        return self.make_dataset(self.train_data)
    
    @property
    def val(self):
        return self.make_dataset(self.val_data)
    
    @property
    def test(self):
        return self.make_dataset(self.test_data)


def create_pinn_model(input_shape, n1, n2, n3, reg):
    def custom_loss_wrapper(model):
        def loss(y_true, y_pred):
            # Gradient computation and loss calculation
            grads_u = K.gradients(model.output[:,:,0], model.input)[0]
            grads_h = K.gradients(model.output[:,:,1], model.input)[0]
            
            du_dx = grads_u[:,:,0]
            du_dt = grads_u[:,:,1]
            dh_dx = grads_h[:,:,0]
            
            g = K.constant(9.8)
            fric_coeff = model.input[:,:,3]
            slope = model.input[:,:,4]
            
            # Saint-Venant equation loss
            loss_saint_venant = du_dt + y_pred[:,:,0] * du_dx + g*dh_dx + g*slope + \
                                g*K.square(fric_coeff) * K.square(y_true[:,:,0]) / \
                                (K.pow(y_true[:,:,1], 4/3) + K.epsilon())
            
            l = K.mean(K.square(loss_saint_venant))
            return 2*l + K.sum(K.mean(K.square(y_pred - y_true), axis=0))
        return loss

    model = Sequential([
        Dense(n1, activation='relu', kernel_regularizer=l2(reg), input_shape=input_shape),
        Dense(n2, activation='relu', kernel_regularizer=l2(reg)),
        Dense(n3, activation='relu', kernel_regularizer=l2(reg)),
        Dense(2)
    ])
    
    custom_loss = custom_loss_wrapper(model)
    
    model.compile(
        optimizer='adam', 
        loss=custom_loss, 
        metrics=['mape', 'mae', 'mse', NSE, r_square]
    )
    
    return model

# Setup and training

'''
The labels (Y variables) are typically handled separately during model training with model.fit(X_train, Y_train). 
The WindowGenerator is responsible for creating input sequences, while the corresponding labels are passed separately during model training.
'''
window = WindowGenerator(
    input_width=6, 
    label_width=6, 
    shift=1, 
    train_data=X_train_2016_2017_pinn, 
    val_data=X_val_2016_2017_pinn, 
    test_data=X_test_2016_2017_pinn
)

X_train_2016_2017_pinn, Y_train_2016_2017_pinn = window.train
X_val_2016_2017_pinn, Y_val_2016_2017_pinn = window.val
X_test_2016_2017_pinn,Y_test_2016_2017_pinn = window.test

# Create model with input shape from windowed data
model = create_pinn_model(
    input_shape=(6, 5),  # 6 time steps, 5 features
    n1=64, 
    n2=32, 
    n3=16, 
    reg=0.001
)

# Training

early_stopping = EarlyStopping(patience=2, verbose=False)
history = model.fit(
    X_train_2016_2017_pinn, Y_train_2016_2017_pinn, 
    epochs=20, 
    batch_size=128, 
    validation_data=(X_val_2016_2017_pinn, Y_val_2016_2017_pinn), 
    callbacks=[early_stopping]
)
'''
# Prediction
def predict_future(model, initial_input, num_steps):
    current_input = initial_input
    predictions = []
    
    for _ in range(num_steps):
        prediction = model.predict(current_input)
        predictions.append(prediction[0, -1])
        
        current_input = np.roll(current_input, -1, axis=1)
        current_input[0, -1] = prediction[0, -1]
    
    return np.array(predictions)

# Predict from last test sequence
last_input = X_test[-1:, :6, :]
future_predictions = predict_future(model, last_input, num_steps=24)

'''